In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
select * from playground.datathon.results

In [ ]:
select * from playground.datathon.drivers

In [ ]:
select * from playground.datathon.races

In [ ]:
select * from playground.datathon.constructors

In [ ]:
select * from playground.datathon.lap_times

In [ ]:
select * from playground.datathon.qualifying

In [ ]:
SELECT *
FROM PLAYGROUND.DATATHON.RESULTS rs
left join playground.datathon.RACES rc on rs.raceid = rc.raceid
--left join playground.datathon.status s on rs.statusid = s.statusid
left join playground.datathon.constructors c on rs.constructorid = c.constructorid
left join playground.datathon.drivers d on rs.driverid = d.driverid


Data Cleaning and pre-processing

In [ ]:
#Calculate position delta
df=raw_race_data.to_pandas()
df['position_delta'] = df['GRID'] - pd.to_numeric(df['POSITION'], errors='coerce')
df.head(10)

#Handle missing values
df= df.dropna(subset=['POSITION'])
df.head(10)



Visual EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(
    x='NAME_1',
    y='position_delta',
    data=df[df['NAME_1'].isin(['Mercedes','Red Bull'])]
)
plt.title('GRID vs FINISH POSITION : Mercedes vs Redbull')
plt.show()

Correlation Model - Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

features=['GRID','POINTS','LAPS']
df=df.dropna(subset=features)
X=df[features]
y=df['POSITION'].astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
model=RandomForestRegressor()
model.fit(X_train,y_train)


#print("RMSE:", mean_squared_error(y_test, model.predict(X_test), squared=False))

rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
print("RMSE:", rmse)
